# Tamas Flesch Thesis - EDA step

## LMJU - UpGrad - DS

### Fifa 23 Ultimate Team player price prediction based on the player's attributes

<span style="color:red">*UPDATE*</span>

The document was updated with the latest EA FC 24 dataset to be able to compare the results with the previous FIFA 23 dataset.

Table of contents
1. Read data
2. Clean data
    - Price_Variation
        - convert the field to numeric
    - Height (extract height in cm into a new column)
        - a few rows were droped because of missing data
    - BodyType (extrct 2 new column, body type text and weight in kg)
        - a few rows were droped because of missing data
    - Club (create new column for ICON players 1/0)
    - League
        - kept the 5 major european league, MLS and 2 special league (ICONS, World cup)
    - Nation
        - kept the 4 country with the most player count which almost have 1000+ players
    - Card_Version
        - removed the column, has mixed values
    - Position columns
        - created 3 separate columns for the alternate positions (those could be NaN if there are no alt positions)
3. Categorical dummy columns
    - create dummy category columns for the normal categorical columns
    - handle alt position columns, these are have missing values as well
    - UPDATE FOR EA FC 24: new column, gender
4. Export the final dataset
5. visualization TODO!!!
    - outliers check
    - correlation matrix
    - https://towardsdatascience.com/an-extensive-guide-to-exploratory-data-analysis-ddd99a03199e

### 1. Read data

#### Imports

In [1]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Pandas and NumPy
import pandas as pd, numpy as np

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# Importing all datasets
futbin_data = pd.read_csv("eafc24_players_2024-06-07.csv")
futbin_data.head()

#### check and understand the Price data column

In [5]:
futbin_data.Price.describe()

In [6]:
len(futbin_data[futbin_data.Price == 0])

In [7]:
futbin_data[futbin_data.Price == 0].head()

SBC concept, ingame, there is a mini game, where players can finish squad building challenges (SBCs), and get valuable player. These players are not in the market, so players can't buy them on the market, that's why they havn't got a price. So I'll remove these cards from the dataset.

In [8]:
futbin_data = futbin_data[futbin_data.Price > 0]

In [9]:
len(futbin_data[futbin_data.Price == 0])

### 2. Clean data

We need only numeric data, so first of all I'll need to investigate the columns, extract the numeric fields, like the height or the body weight. 

Then create dummy columns for the categorical columns like the run style. 

At the last step, I'll create some derived columns where I need to merge some information from columns with a lot of different text values. Like the club or the card version. These columns have a lot of different, distinct values, so I will select some of the most valueable categories and will create a special coulmn with 1/0 values. 1 for the most special categories and 0 for the other columns.

#### Price_Variation

In [10]:
futbin_data.Price_Variation.value_counts()

too much missing valuees, and based on the differences, it seems to me that this column has a lot of outliners as well, I think it is better to remove it from the dataset.

In [11]:
futbin_data = futbin_data.drop(['Price_Variation'], 1)

#### Height

In [12]:
futbin_data.head()["Height"][0]

In [13]:
futbin_data.Height.info()

In [14]:
futbin_data.Height.value_counts()

In [15]:
# remove 0 values, 4 records

In [16]:
futbin_data.shape[0]

In [17]:
futbin_data[futbin_data.Height == "0"]

In [18]:
futbin_data = futbin_data.drop(futbin_data[futbin_data.Height == "0"].index)

In [19]:
futbin_data.shape[0]

In [20]:
futbin_data[futbin_data.Height == "0"]

In [21]:
futbin_data['Height_in_cm'] = futbin_data['Height'].str[:3]

In [22]:
futbin_data['Height_in_cm'].info()

In [23]:
futbin_data['Height_in_cm'] = pd.to_numeric(futbin_data['Height_in_cm'])

In [24]:
futbin_data.info()

#### Body type

In [25]:
futbin_data.BodyType.info()

In [26]:
futbin_data.BodyType.value_counts()

I need to create two more columns from this one. The first one would be a numeric field in kg. The second is a categorical field with the body type.

In [27]:
futbin_data[['BodyType_Text', 'BodyType_Weight']] = futbin_data['BodyType'].str.split('  ', 1, expand=True)

First try, using '  ' (double spaces), let's try, is it working?

In [28]:
futbin_data.info()

Some rows are missing from the Weight coulmn, maybe double-space is not the right separator for every row.

In [29]:
futbin_data.BodyType_Text.value_counts()

In [30]:
futbin_data.BodyType_Weight.value_counts()

Second try, sometimes we have 2 or 3 spaces as a separator so I will try a parenthesis (

In [31]:
futbin_data[['BodyType_Text', 'BodyType_Weight']] = futbin_data['BodyType'].str.split('(', 1, expand=True)

In [32]:
futbin_data.info()

In [33]:
futbin_data.BodyType_Text.value_counts()

In [34]:
futbin_data.BodyType_Weight.value_counts()

In [35]:
futbin_data.BodyType_Weight.count()

In [36]:
futbin_data[futbin_data.BodyType_Weight.isnull() == True]

In [37]:
futbin_data = futbin_data.drop(futbin_data[futbin_data.BodyType_Weight.isnull() == True].index)

In [38]:
futbin_data.info()

In [39]:
futbin_data.BodyType_Text.value_counts()

In [40]:
futbin_data.BodyType_Text = futbin_data.BodyType_Text.str.strip()

In [41]:
futbin_data.BodyType_Weight.value_counts()

In [42]:
futbin_data.BodyType_Weight = futbin_data.BodyType_Weight.str[:-3]

In [43]:
futbin_data.BodyType_Weight = pd.to_numeric(futbin_data.BodyType_Weight)

In [44]:
futbin_data.info()

In [45]:
futbin_data.head()

Remove extracted columns

In [46]:
futbin_data = futbin_data.drop(['Height','BodyType'], 1)

In [47]:
futbin_data.Club.value_counts()

#### Club

It has many values, I'll create a new column which will have 1 if the club is FUT ICONS or HERO and 0 otherwise.

In [48]:
# Defining the map function
def icon_map(x):
    if x == 'EA FC ICONS':
        return 1
    elif x == 'HERO':
        return 1
    else:
        return 0

# Applying the function to the club column
futbin_data['Club_Hero'] = futbin_data['Club'].apply(icon_map)

In [49]:
futbin_data.Club_Hero.value_counts()

In [50]:
futbin_data = futbin_data.drop(['Club'], 1)

In [51]:
futbin_data.head()

#### League

In [52]:
futbin_data.League.value_counts()

We have many leagues, I'll collect the 5 mayor european leagues, and some special cases (Worls Cup, Icons), and all of the other values will be other. After this simplification I'll create categorical columns from the values.

In [53]:
# Defining the map function
def special_club_map(x):
    if x == 'Icons':
        return 'Icons'
    elif x == 'LALIGA EA SPORTS':
        return 'LALIGA EA SPORTS'
    elif x == 'Premier League':
        return 'Premier League'
    elif x == 'MLS':
        return 'MLS'
    elif x == 'Serie A TIM':
        return 'Serie A TIM'
    elif x == 'Ligue 1 Uber Eats':
        return 'Ligue 1 Uber Eats'
    elif x == 'Bundesliga':
        return 'Bundesliga'
    else:
        return 'Other'

# Applying the function to the club column
futbin_data['League_Cat'] = futbin_data['League'].apply(special_club_map)

In [54]:
futbin_data.League_Cat.value_counts()

In [55]:
futbin_data = futbin_data.drop(['League'], 1)

In [56]:
futbin_data.head()

#### Nation

I'll check the nation column, but I have a feeling that I need to remove it, because it has probably a lot of values and it will be hard to extract information from it.

In [57]:
futbin_data.Nation.value_counts()

We have a few nations more than 1000 players so I decided that I will keep those, and will use other for the others. It is just a subjective decision, so later it can be modified, and maybe keep other nations as well.

One other derived direction could be to merge the countries by continents Europe, America, Asia, and so on...

In [58]:
# Defining the map function
def nation_keep_map(x):
    if x == 'England':
        return 'England'
    elif x == 'Germany':
        return 'Germany'
    elif x == 'Spain':
        return 'Spain'
    elif x == 'France':
        return 'France'
    elif x == 'Argentina':
        return 'Argentina'
    elif x == 'United States':
        return 'United States'
    elif x == 'Italy':
        return 'Italy'
    elif x == 'Brazil':
        return 'Brazil'
    elif x == 'Holland':
        return 'Holland'
    elif x == 'Portugal':
        return 'Portugal'
    else:
        return 'Other'

# Applying the function to the club column
futbin_data['Nation_Cat'] = futbin_data['Nation'].apply(nation_keep_map)

In [59]:
futbin_data.Nation_Cat.value_counts()

In [60]:
futbin_data = futbin_data.drop(['Nation'], 1)

In [61]:
futbin_data.head()

#### Card_Version

In [62]:
futbin_data.Card_Version.value_counts()

too many options and the values are mixed, not just card versions (toty, fut hero, winter wildcards, etc) but positions (LB, RB, etc and these combinations as well)

I'll just remove this column for now.

In [63]:
futbin_data = futbin_data.drop(['Card_Version'], 1)

In [64]:
futbin_data.head()

#### Positions

In [65]:
futbin_data.Main_Position.value_counts()

would be perfect for categorical columns, will convert it at the last step.

In [66]:
futbin_data.Alternate_Positions.value_counts()

In [67]:
futbin_data['Alt_Pos_List'] = futbin_data['Alternate_Positions'].str.split(',')

In [68]:
futbin_data['Alt_Pos_List'].head()

In [69]:
#futbin_data[['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3']] = futbin_data['Alternate_Positions'].str.split(',', 1, expand=True)

In [70]:
futbin_data = futbin_data.join(futbin_data['Alternate_Positions'].str.split(',', expand=True).rename(columns={0:'Alt_Pos_1', 1:'Alt_Pos_2', 2:'Alt_Pos_3'}))

In [71]:
futbin_data.head()

In [72]:
futbin_data['Alt_Pos_Count'] = futbin_data['Alt_Pos_List'].str.len()

In [73]:
futbin_data = futbin_data.drop(['Alternate_Positions'], 1)

In [74]:
futbin_data = futbin_data.drop(['Alt_Pos_List'], 1)

In [75]:
futbin_data.head()

In [76]:
futbin_data.tail()

In [77]:
futbin_data.Alt_Pos_1.replace('0', np.nan, inplace=True)

In [78]:
futbin_data.Alt_Pos_2.fillna(value=np.nan, inplace=True)

In [79]:
futbin_data.Alt_Pos_3.fillna(value=np.nan, inplace=True)

Need to update Alt_Pos_Count to 0 if the Alt_Pos_1 is NaN, because originally, the field conatins 0 instead of null value.

In [80]:
#futbin_data['Alt_Pos_Count'] = futbin_data.apply(lambda x: 0 if x['Alt_Pos_1']=='NaN' else x['Alt_Pos_Count'], axis=1)
futbin_data.Alt_Pos_Count = np.where(futbin_data.Alt_Pos_1.isna(), 0, futbin_data.Alt_Pos_Count)

In [81]:
futbin_data.Alt_Pos_1.info()

In [82]:
futbin_data[futbin_data.Alt_Pos_1.isna()].tail()

In [83]:
futbin_data.tail()

### 3. Categorical columns

Main_Position

Run_Style

Attack_Workrate

Defense_Workrate

BodyType_Text

League_Cat

Nation_Cat

======================================================

Alt_Pos_1

Alt_Pos_2

Alt_Pos_3

First I'll handle the "normal" cat columns, skipping the from alt pos, because those could have emty fields.

In [84]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy_fields = pd.get_dummies(futbin_data[['Main_Position', 'Run_Style', 'Attack_Workrate', 'Defense_Workrate', 
                                           'BodyType_Text', 'League_Cat', 'Nation_Cat']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_fields], axis=1)

In [85]:
futbin_data.head()

In [86]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Main_Position', 'Run_Style', 'Attack_Workrate', 
                        'Defense_Workrate', 'BodyType_Text', 'League_Cat', 'Nation_Cat'], 1)

In [87]:
futbin_data.head()

#### Alt positins filling with missing value, and use it as a category

##### Alt_Pos_1

In [88]:
futbin_data.Alt_Pos_1.isnull().sum()

In [89]:
futbin_data.Alt_Pos_1.fillna('missing', inplace=True)

In [90]:
futbin_data.Alt_Pos_1.isnull().sum()

In [91]:
futbin_data.Alt_Pos_1.value_counts()

##### Alt_Pos_2

In [92]:
futbin_data.Alt_Pos_2.isnull().sum()

In [93]:
futbin_data.Alt_Pos_2.fillna('missing', inplace=True)

In [94]:
futbin_data.Alt_Pos_2.isnull().sum()

In [95]:
futbin_data.Alt_Pos_2.value_counts()

##### Alt_Pos_3

In [96]:
futbin_data.Alt_Pos_3.isnull().sum()

In [97]:
futbin_data.Alt_Pos_3.fillna('missing', inplace=True)

In [98]:
futbin_data.Alt_Pos_3.isnull().sum()

In [99]:
futbin_data.Alt_Pos_3.value_counts()

In [100]:
futbin_data.head()

#### Create categorical columns for the alt pos columns

In [101]:
# Creating a dummy variable for the alt_pos categorical variables and dropping the first one.
dummy_altpos = pd.get_dummies(futbin_data[['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_altpos], axis=1)

In [102]:
futbin_data.head()

In [103]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Alt_Pos_1', 'Alt_Pos_2', 'Alt_Pos_3'], 1)

In [104]:
futbin_data.head()

#### Gender column handling

In [105]:
# Creating a dummy variable for some of the categorical variables and dropping the first one.
dummy_fields = pd.get_dummies(futbin_data[['Gender']], drop_first=True)

# Adding the results to the master dataframe
futbin_data = pd.concat([futbin_data, dummy_fields], axis=1)

In [106]:
futbin_data.head()

In [107]:
# dropping the repeated variables
futbin_data = futbin_data.drop(['Gender'], 1)

In [108]:
futbin_data.head()

In [109]:
futbin_data.Gender_Male.value_counts()

In EA FC 24, Female players were introduced, there are 1302 female players next to the 7695 male players in the dataset.

### 4. Export final dataset

In [110]:
csv_futbin = futbin_data.to_csv('futbin24.csv', index = False)
print('\nCSV String:\n', csv_futbin)